# Mixtral-8x7B稀疏大模型最佳实践

本方案旨在帮助大模型开发者快速上手平台，实现稀疏大语言模型Mixtral的高效分布式训练、三阶段指令微调、模型离线推理和在线服务部署等完整的开发流程。以Mixtral-8x7B模型为例，为您详细介绍该方案的开发流程。

## 步骤一：准备Mixtral-8x7B-v0.1

1. 从ModelScope社区下载权重，安装下载工具

In [ ]:
! pip install modelscope

2. 执行以下命令下载示例代码：

In [ ]:
# ### Loading Model and Tokenizer
from modelscope.hub.snapshot_download import snapshot_download
model_dir = snapshot_download('AI-ModelScope/Mixtral-8x7B-v0.1', 'v1.1.4')
# 获取下载路径
print(model_dir)
# model_dir = snapshot_download('AI-ModelScope/Mixtral-8x7B-Instruct-v0.1', 'v1.1.4')

3. 控制台执行以下命令

    ```basj
    # mkdir -p /mnt/workspace/mixtral-ckpts/${后缀为hf的ckpt文件夹}
    mkdir -p /mnt/workspace/mixtral-ckpts/Mixtral-8x7B-v0.1
    # cp -r ${在此处填写已下载的模型路径}/* /mnt/workspace/mixtral-ckpts/${后缀为hf的ckpt文件夹}。
    cp -r /root/.cache/modelscope/hub/mixtral/Mixtral-8x7B-v0.1/* /mnt/workspace/mixtral-ckpts/Mixtral-8x7B-v0.1
    ```

## 步骤二：准备预训练数据
建议您在DSW实例中准备预训练数据。本案例以WuDaoCorpora2.0数据集（该数据集仅供研究使用）为例，介绍Megatron训练数据的预处理流程。

    ```bash
    mkdir /mnt/workspace/mixtral-datasets/
    cd /mnt/workspace/mixtral-datasets
    wget https://atp-modelzoo-wlcb-pai.oss-cn-wulanchabu.aliyuncs.com/release/models/pai-megatron-patch/mistral-datasets/alpaca_zh-mistral-train.json
    wget https://atp-modelzoo-wlcb-pai.oss-cn-wulanchabu.aliyuncs.com/release/models/pai-megatron-patch/mistral-datasets/alpaca_zh-mistral-valid.json
    mkdir -p /mnt/workspace/mixtral-datasets/wudao
    cd /mnt/workspace/mixtral-datasets/wudao
    wget https://atp-modelzoo-wlcb-pai.oss-cn-wulanchabu.aliyuncs.com/release/models/pai-megatron-patch/mistral-datasets/wudao_mistralbpe_content_document.idx
    wget https://atp-modelzoo-wlcb-pai.oss-cn-wulanchabu.aliyuncs.com/release/models/pai-megatron-patch/mistral-datasets/wudao_mistralbpe_content_document.bin
    ```

## 步骤三：Megatron训练
您可以按照以下流程进行Megatron训练。

### 模型格式转换
将Huggingface格式的模型文件转换为Megatron格式。

    ```
    cd /mnt/workspace/
    mkdir mixtral-ckpts
    cd mixtral-ckpts
    wget https://atp-modelzoo-wlcb-pai.oss-cn-wulanchabu.aliyuncs.com/release/models/pai-megatron-patch/mistral-ckpts/Mixtral-8x7B-v0.1-to-mcore-tp4-ep4.tar.zst
    tar -zxf Mixtral-8x7B-v0.1-to-mcore-tp4-ep4.tar.zst
    ```

### 预训练模型
您可以在DSW单机环境中训练模型，也可以在DLC环境中提交多机多卡分布式训练任务，训练过程大约持续2个小时。任务执行成功后，模型文件将输出到/mnt/workspace/output_megatron_mixtral/目录下。

## 步骤四：模型格式转换
您可以将训练获得的Megatron格式的模型转换为Huggingface格式，具体操作步骤如下。后续您可以使用转换后的Huggingface格式的模型进行服务在线部署。

## 步骤五：部署及调用模型服务
完成离线推理并评估完成模型效果后，您可以将转换为Huggingface格式的模型部署为在线服务，并在实际的生产环境中调用，从而进行推理实践。具体操作步骤如下：